## Задание 1

Опеределите части речи слов в этом тексте, используя пайморфи. В форму запишите слова, которым приписан тэг VERB. 

In [2]:
text = "Зонды будут запущены в космос в 2029 году и отправятся ко второй точке Лагранжа в системе Солнце"

In [5]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

for word in text.split():
    parsed = morph.parse(word)[0].tag.POS
    if parsed == 'VERB':
        print(word)

будут
отправятся


## Задание 2

На данных предложениях обучите TFIDF векторайзер из sklearn (с дефолтными параметрами). Найдите слово с самым высоким коэффициентом tfidf в первом тексте. Вставьте его в форму.

In [6]:
import requests
r = requests.get(
    'https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/anna_karenina.txt'
)

# работайте с этими предложениями
sentences = r.text.split('\n')
sentences = [sent for sent in sentences if len(sent) > 10]

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer().fit(sentences)

In [32]:
import numpy as np

np.argmax(vec.transform([sentences[0]]).toarray())

1962

In [34]:
vec.get_feature_names()[1962]

'буржуазного'

## Задание 3

Токенизируйте текст Анны Карениной с помощью библиотеки раздел, привидите все к нижнему регистру. Рассчитайте статистики биграммов и униграммов. Найдите наиболее вероятное продолжение "красный ...". Вставьте наиболее веротяное следующее слово в форму.

In [74]:
import razdel
from collections import Counter
from gensim.models.phrases import Phrases
from nltk.tokenize import sent_tokenize


def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams


unigrams_bag = Counter()
word_lst = []
for sent in sentences:
    for word in razdel.tokenize(sent):
        unigrams_bag.update([word.text.lower()])
        
cnt_words = sum(list(unigrams_bag.values()))
unigram_probas = Counter({word:c / cnt_words for word, c in unigrams_bag.items()})

bigrams_bag = Counter()
for sent in sentences:
    sent_tokenized = [el.text.lower() for el in razdel.tokenize(sent)]
    bigrams = ngrammer(sent_tokenized)
    bigrams_bag.update(bigrams)
    
cnt_bigrams = sum(list(bigrams_bag.values()))
bigram_probas = Counter({word:c / cnt_bigrams for word, c in bigrams_bag.items()})

In [83]:
matrix = np.zeros((len(unigrams_bag), len(unigrams_bag)))
id2word = list(unigrams_bag.keys())
word2id = {word:i for i, word in enumerate(id2word)}

for ngram in bigram_probas:
    word_1, word_2 = ngram.split()
    matrix[word2id[word_1]][word2id[word_2]] = bigram_probas[ngram] / unigram_probas[word_1]

In [82]:
id2word[np.argmax(matrix[word2id['красный']])]

'мешочек'

In [89]:
import re

{el: value for el, value in bigrams_bag.items() if re.findall('\\bкрасный\\b', el)}

{'заперла красный': 1,
 'красный мешочек': 5,
 'держа красный': 1,
 'потом красный': 1,
 'красный огонь': 1,
 ', красный': 1,
 'красный ,': 2,
 'лбом красный': 1,
 'красный платок': 1,
 'навстречу красный': 1,
 'перекатывавший красный': 1,
 'красный .': 1,
 'маленький красный': 1,
 'но красный': 1,
 'откинула красный': 1}

## Задание 4

Расчитайте ненормализованное растояние Дамерау-Левенштейна и ненормализованное растояние Левенштейна между этими словами (каждое с каждым), найдите пару слов, для которых эти расстояния отличаются. Вставьте эти слова в форму

In [95]:
words = ["решение","ршеение","ренешик","рещиние","ришение"]

In [96]:
import textdistance

In [100]:
for word_1 in words:
    for word_2 in words:
        print(word_1, word_2, textdistance.levenshtein(word_1, word_2), textdistance.damerau_levenshtein(word_1, word_2), textdistance.levenshtein(word_1, word_2) == textdistance.damerau_levenshtein(word_1, word_2))

решение решение 0 0 True
решение ршеение 2 1 False
решение ренешик 3 3 True
решение рещиние 2 2 True
решение ришение 1 1 True
ршеение решение 2 1 False
ршеение ршеение 0 0 True
ршеение ренешик 4 4 True
ршеение рещиние 3 3 True
ршеение ришение 2 2 True
ренешик решение 3 3 True
ренешик ршеение 4 4 True
ренешик ренешик 0 0 True
ренешик рещиние 4 4 True
ренешик ришение 4 4 True
рещиние решение 2 2 True
рещиние ршеение 3 3 True
рещиние ренешик 4 4 True
рещиние рещиние 0 0 True
рещиние ришение 3 3 True
ришение решение 1 1 True
ришение ршеение 2 2 True
ришение ренешик 4 4 True
ришение рещиние 3 3 True
ришение ришение 0 0 True


## Задание 5

Загрузите токенизатор distilbert-base-uncased из huggingface. Токенизируйте предложения из Анны Карениной с помощью предобученного токенизатора и обучите Fastext модель со следующими параметрами: cbow, размер вектора 300, размер окна 5, минимальный размер символьного нграмма - 2, максимальный размер символьного нграмма - 7. Найдите ближайшее слово к слову "каренина", вставьте его в форму.

In [101]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [108]:
def decode_bert_tokenizer(text):
    encoded = tokenizer.encode(text)
    return [tokenizer.decode([x]) for x in encoded]

In [109]:
decode_bert_tokenizer('Привет мир!!!!')

['[CLS]',
 'п',
 '##р',
 '##и',
 '##в',
 '##е',
 '##т',
 'м',
 '##и',
 '##р',
 '!',
 '!',
 '!',
 '!',
 '[SEP]']

In [110]:
corpus = []
for sent in sentences:
    corpus.append(decode_bert_tokenizer(sent))

Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors


In [127]:
corpus[:10]

[['[CLS]',
  '«',
  'а',
  '##н',
  '##на',
  'к',
  '##а',
  '##р',
  '##е',
  '##н',
  '##и',
  '##на',
  '»',
  '–',
  'э',
  '##т',
  '##о',
  'с',
  '##л',
  '##о',
  '##ж',
  '##н',
  '##о',
  '##е',
  ',',
  'п',
  '##с',
  '##и',
  '##х',
  '##о',
  '##л',
  '##о',
  '##г',
  '##и',
  '##ч',
  '##е',
  '##с',
  '##к',
  '##и',
  'у',
  '##т',
  '##о',
  '##н',
  '##ч',
  '##е',
  '##н',
  '##н',
  '##о',
  '##е',
  ',',
  'о',
  '##с',
  '##т',
  '##р',
  '##о',
  '##п',
  '##р',
  '##о',
  '##б',
  '##л',
  '##е',
  '##м',
  '##н',
  '##о',
  '##е',
  'п',
  '##р',
  '##о',
  '##и',
  '##з',
  '##в',
  '##е',
  '##д',
  '##е',
  '##н',
  '##и',
  '##е',
  ',',
  'н',
  '##а',
  '##с',
  '##ы',
  '##щ',
  '##е',
  '##н',
  '##н',
  '##о',
  '##е',
  'п',
  '##р',
  '##и',
  '##м',
  '##е',
  '##т',
  '##а',
  '##м',
  '##и',
  'в',
  '##р',
  '##е',
  '##м',
  '##е',
  '##н',
  '##и',
  '.',
  'л',
  '.',
  'н',
  '.',
  'т',
  '##о',
  '##л',
  '##с',
  '##т',
  '##о',
  '##и'

In [118]:
from gensim.models import FastText
from gensim.models.callbacks import CallbackAny2Vec

class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1
        
epoch_logger = EpochLogger()

model = FastText(vector_size=300, window=5, sg=0, min_n=2, max_n=7)
model.build_vocab(corpus)
model.train(corpus, total_examples=len(corpus), epochs=10, callbacks=[epoch_logger])

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end


(4052400, 13888640)

In [128]:
model.wv.similar_by_word('каренина')

[('##ка', 0.5696330070495605),
 ('а', 0.49767211079597473),
 ('##на', 0.40630394220352173),
 ('##ович', 0.36692699790000916),
 ('–', 0.2744726538658142),
 ('к', 0.2697364389896393),
 ('ф', 0.2500544488430023),
 ('с', 0.20072151720523834),
 ('з', 0.1963053047657013),
 ('[SEP]', 0.18371255695819855)]